In [ ]:
#include "../helpers/matrix_functions.h"
#include "../helpers/gauss_functions.h"
#include <stdbool.h>
#include <mpi.h>
#pragma cling load("libmpi.so")

In [ ]:
// Gaussian elimination (Solve systems of linear equations)
// start_main
int rank, p;
// Global input matrix and vector with shape [X|x]
double *B;
// Global output matrix and vector with shape [X|x]
double *A;
// Local matrix and vector with shape [X|x]
double *a;

// Global matrix size
const int n = 64;
int m = n + 1;

// Number of rows in local matrix a
int l;

double *pivot_row = (double *)malloc(m * sizeof(double));
int pivot_idx;

// start_timing
MPI_Init(NULL, NULL);

MPI_Comm_rank(MPI_COMM_WORLD, &rank);
MPI_Comm_size(MPI_COMM_WORLD, &p);

// start_init

if (rank == 0) {

    init_matrix(&B, n, m, false);

    A = (double *)malloc(n * m * sizeof(double));

    pivot_idx = 0;

    for (int r = p - 1; r >= 0; r--) {
        // Determine the size of the local matrix
        l = n / p + (r < n % p ? 1 : 0);

        // Initialize local matrix
        a = (double *)malloc(l * m * sizeof(double));

        for (int loc_row = 0; loc_row < l; loc_row++) {
            for (int j = 0; j < m; j++) {
                int glob_row = r + loc_row * p;
                a[loc_row * m + j] = B[glob_row * m + j];
            }
        }
        if (r > 0) {
            MPI_Send(&l, 1, MPI_INT, r, 0, MPI_COMM_WORLD);
            MPI_Send(a, l * m, MPI_DOUBLE, r, 1, MPI_COMM_WORLD);
        }
    }
}

if (rank != 0) {
    MPI_Recv(&l, 1, MPI_INT, 0, 0, MPI_COMM_WORLD, MPI_STATUS_IGNORE);
    a = (double *)malloc(l * m * sizeof(double));

    MPI_Recv(a, l * m, MPI_DOUBLE, 0, 1, MPI_COMM_WORLD, MPI_STATUS_IGNORE);
}
// end_init

// start_compute

// Elimination
for (int k = 0; k < n - 1; k++) {
    int local_row = (k + 1 - rank) / p;
    // Processor with rank k%p gets the pivot line.
    // This line must be distributed to all other processors.
    if (rank == k % p) {
        for (int i = 0; i < m; i++) {
            pivot_row[i] = a[i + local_row * m];
        }
    }

    MPI_Bcast(&(pivot_row[k]), m - k, MPI_DOUBLE, k % p, MPI_COMM_WORLD);

    // The pivot line will not be changed anymore.
    // Therefore it can already be copied into the result matrix
    if (rank == 0) {
        for (int j = 0; j < m; j++) {
            int idx = pivot_idx * m + j;
            if (j < k) {
                A[idx] = 0;
            } else {
                A[idx] = pivot_row[j];
            }
        }
        pivot_idx++;
    }

    for (int i = local_row; i < l; i++) {
        // rank+i*p>=k+1 <-> i>=(k+1-rank)/p
        double q = a[i * m + k] / pivot_row[k];
        for (int j = k; j < m; j++)
            a[i * m + j] -= q * pivot_row[j];
    }
}

// end_compute

// start_check

if (rank == (n - 1) % p) {
    // printf("Überprüfung des Ergebnisses\n");
    // Die letzte Pivotzeile wird vom Prozessor mit Rang (n-1)%p an den
    // Prozessor mit Rang 0 geschickt
    int local_row = (n - rank) / p;
    for (int i = 0; i < m; i++) {
        pivot_row[i] = a[i + local_row * m];
    }
}
// MPI_Bcast(&(pivot_row[n-1]),m-n+1,MPI_DOUBLE,(n-1)%p,MPI_COMM_WORLD);
if (rank == (n - 1) % p) {
    MPI_Send(&(pivot_row[n - 1]), m - n + 1, MPI_DOUBLE, 0, 2,
             MPI_COMM_WORLD);
} else if (rank == 0) {
    MPI_Recv(&(pivot_row[n - 1]), m - n + 1, MPI_DOUBLE, (n - 1) % p, 2,
             MPI_COMM_WORLD, MPI_STATUS_IGNORE);
}
free(a);
// Check result
// The last pivot line is copied to the output matrix
if (rank == 0) {
    for (int j = 0; j < m; j++) {
        int idx = pivot_idx * m + j;
        if (j < n - 1) {
            A[idx] = 0;
        } else {
            A[idx] = pivot_row[j];
        }
    }
    pivot_idx++;

    // print_matrix_vector(A,n,m);

    // Solution vector
    double *x = (double *)malloc(n * sizeof(double));
    backsub(A, n, m, x);
    double error = check(A, n, m, x);
    // printf("Error: %f\n", error);
    free(A);
    free(B);
    free(x);
}

// end_check

MPI_Finalize();

// end_timing
// end_main